In [176]:
import os
import pandas as pd
import datetime
import pickle

# Data Processing Steps

First, you are asked to write code to perform the following data processing steps:
1. Converting the EOD data into five separate time series data frames; one
each for Open, High, Low, Close and Volume. In each data frame, rows
should be indexed by date, and columns by ticker.
2. Create a data frame containing the future close returns as defined by Eq.
(1), in the same format. Also create a data frame containing close returns.
3. Create a data frame containing the ratios of High
Low
for each ticker each
day, in the same format.

In [28]:
# Read all csv files and append them into one dataframe.

path ="ASX-2015-2018"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        filelist.append(os.path.join(root,file))

        
list_of_dataframes = []
#print allhe file names
for name in filelist:
    list_of_dataframes.append(pd.read_csv(name, index_col="Date", parse_dates=True, names=["Ticker", "Date", "Open", "High", "Low", "Close", "Volume"]))
    
merged_df = pd.concat(list_of_dataframes)

In [178]:
merged_df.to_pickle("merged.pkl")

## Step 1

In [135]:
open_df = merged_df[["Ticker", "Open"]]
high_df = merged_df[["Ticker", "High"]]
low_df = merged_df[["Ticker", "Low"]]
close_df = merged_df[["Ticker", "Close"]]
volume_df = merged_df[["Ticker", "Volume"]]

In [136]:
volume_df

,Ticker,Volume
Date,,
2015-01-02,1PG,607639
2015-01-02,3PL,1863
2015-01-02,8IH,112700
2015-01-02,AAA,409863
2015-01-02,AAC,90150
...,...,...
2018-06-29,XTJ,0
2018-06-29,XTL,0
2018-06-29,XTO,0


In [137]:
def unique_tickers(df, data_type):
    #Return a list of unique tickers

    #Parameters:
    #    df (pandas DataFrame):Dataframe with combined tickers
    #    data_type (str):Data description in the df column

    #Returns:
    #    uniquedf_list (list):List with unique ticker df
    
    
    tickers = pd.unique(df['Ticker'])
    uniquedf_list = []
    for ticker in tickers:
        unique_df = df[df['Ticker'] == ticker]
        unique_df = pd.DataFrame(unique_df[data_type].rename(ticker))
        uniquedf_list.append(unique_df)
        
    return uniquedf_list

In [138]:
def concat_dfs(df_list):
    #Return a concatenated by column dataframe

    #Parameters:
    #    df_list (list of pandas DataFrame):List with unique ticker df

    #Returns:
    #    main_df (pd.DataFrame):Concatenated dataframe by column
    main_df = pd.DataFrame()
    for df in df_list:
        main_df = pd.concat([main_df, df], axis=1)
    return main_df

In [139]:
def data_processing(df_list, type_list):
    for index in range(len(df_list)):
        tickerdf_list = unique_tickers(df_list[index], type_list[index])
        
        final_df = pd.DataFrame()
        
        final_df = concat_dfs(tickerdf_list)
        final_df.to_pickle("{}.pkl".format(type_list[index]))

In [140]:
%%time
df_list = [open_df, high_df, low_df, close_df, volume_df]
type_list = ["Open", "High", "Low", "Close", "Volume"]
data_processing(df_list, type_list)

Wall time: 27min 49s


Note: Could have used pandas.Pivot() for future reference.

## Step 2

In [168]:
open_file = open("Open.pkl",'rb')
open_df = pickle.load(open_file)

high_file = open("High.pkl",'rb')
high_df = pickle.load(high_file)

low_file = open("Low.pkl",'rb')
low_df = pickle.load(low_file)

close_file = open("Close.pkl",'rb')## Step 1
close_df = pickle.load(close_file)

volume_file = open("Volume.pkl",'rb')
volume_df = pickle.load(volume_file)

pd.DataFrame.pct_change(1) is the same as the formula as defined by Eq. (1).

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html

In [169]:
future_return1_df = close_df.pct_change(1)

In [170]:
future_return1_df
future_return1_df.to_pickle("future_return1.pkl")

In [171]:
close_return_df = close_df / open_df -1

In [172]:
close_return_df
close_return_df.to_pickle("close_return.pkl")

## Step 3

In [173]:
high_low_df = high_df / low_df

In [174]:
high_low_df
high_low_df.to_pickle("high_low.pkl")

In [ ]:
volume_change_lag1_df = volume_df.pct_change(periods=1)

volume_change_lag1_df = volume_change_lag1_df.shift(periods=1)

volume_change_lag1_df.to_pickle("volume_change_lag1.pkl")